In [1]:
import os
%pwd

'd:\\DL MODEL\\ImageClassificationModel\\research'

## Base Model Preparing

In [2]:
#update the Entity: taking rtn as function
from dataclasses import dataclass
from pathlib import Path

In [3]:
@dataclass
#creating class for entity and defining the class variable init
class PrepareBaseModel_Config():
    root_dir:Path
    base_model_path:Path
    update_base_model_path:Path
    param_image_size:list
    param_learning_rate:float
    param_include_top:bool
    param_weights:str
    param_classes:int


In [4]:
from src.ImageClassifier.Utils import download_data_from_s3,read_yaml_file,Create_Directory
from src.ImageClassifier.Constants import *
from src.ImageClassifier.Exception import CustomException
from src.ImageClassifier.loggers import logger
import os,sys


In [5]:
#updating the configuration Manager class to prepare the basemodel file!!!
class ConfigurationManager():
    #creating constructor method to we used to read the yaml file and create directories accordinbgly
    def __init__(self, config_file_path=CONFIG_FILEPATH,param_filepath=PARAM_FILEPATH):
        #reading the yaml file taking rtn as configbox dictatonary
        self.config = read_yaml_file(config_file_path)
        self.param = read_yaml_file(param_filepath)

        #creating directories
        Create_Directory([self.config.artifacts_root]) #create artifacts directories

    #creating another built in method to initial the base model file
    def get_base_model_config(self) ->PrepareBaseModel_Config:
        config = self.config.prepare_base_model #rtn configbox dictatonary
        print(config)

        #create root directories
        Create_Directory([config.root_dir]) #artifacts/prepare_base_model create this directory

        #creating an object PrepareBaseModel_Config class and passing the value to this parameter
        prepare_base_model_config = PrepareBaseModel_Config(
            root_dir = Path(config.root_dir),
            base_model_path = Path(config.base_model_path),
            update_base_model_path = Path(config.update_base_model_path),
            param_image_size = self.param.IMAGE_SIZE,
            param_learning_rate= self.param.LEARNING_RATE,
            param_include_top= self.param.INCLUDE_TOP,
            param_weights= self.param.WEIGHTS,
            param_classes= self.param.CLASSES,
            
        )

        return prepare_base_model_config

In [6]:
import tensorflow as tf
import zipfile

In [7]:
os.chdir('../')

In [8]:
#update the components the file
class BaseModel():
    def __init__(self,config:PrepareBaseModel_Config):
        self.config = config #initilaizing variable rtn all parameter values which is defined in class variable of PrepareBaseModel_Config class

    #creating another to get base model
    def get_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape=self.config.param_image_size,
            weights=self.config.param_weights,
            include_top=self.config.param_include_top,
            classes=self.config.param_classes
        )
        #now saving the base Model file
        self.save_model(path=self.config.base_model_path,model=self.model)

    @staticmethod #we dont have to define its a class method or instance method
    def _prepare_full_model(model,classes,learning_rate,freeze_all,freeze_till):
        if freeze_all:
            #freezing the convolutional layer to the model
            for layer in model.layers:
                layer.trainable = False #means dont do update weights and bias files keep it same as base model
        elif (freeze_till is not None) and (freeze_till>0):
            for layer in model.layers[:-freeze_till]:
                layer.trainable = False

        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(
            units=classes, activation='softmax')(flatten_in)
        
        full_model = tf.keras.models.Model(
            inputs=model.input, outputs=prediction)
        
        #now compiling the model
        full_model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )

        #to check the summary of model
        full_model.summary()
        
        
        return full_model


    #creating another method to update the model
    def update_base_model(self): #in this model we add dense layer or outputlayer  freezing the layer to sequential model
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes = self.config.param_classes,
            freeze_all = True,
            freeze_till=None,
            learning_rate = self.config.param_learning_rate
        )

        #now saving the updated base Model file
        self.save_model(path=self.config.update_base_model_path,model=self.full_model)

    #creating function to save model
    @staticmethod
    def save_model(path:Path,model:tf.keras.Model):
        #saving the model
        model.save(path)
        

In [9]:
#updating the pipeline
try:
    #creating an object of configuration manager class
    cm = ConfigurationManager()
    prepare_base_model_config = cm.get_base_model_config()

    #creating an object of BaseModel class
    base_model = BaseModel(config=prepare_base_model_config)

    base_model.get_base_model()

    base_model.update_base_model()


    

except Exception as e:
    raise CustomException(e,sys)

[2024-09-07 21:23:37,802]-INFO-66-Reading the YAML file config\config.yaml
[2024-09-07 21:23:37,807]-INFO-69-YAML file read successfully: config\config.yaml
[2024-09-07 21:23:37,834]-INFO-66-Reading the YAML file params.yaml
[2024-09-07 21:23:37,853]-INFO-69-YAML file read successfully: params.yaml
[2024-09-07 21:23:37,855]-INFO-80-Creating Directory
{'root_dir': 'artifacts/prepare_base_model', 'base_model_path': 'artifacts/prepare_base_model/base_model.h5', 'update_base_model_path': 'artifacts/prepare_base_model/base_model_update.h5'}
[2024-09-07 21:23:37,858]-INFO-80-Creating Directory
[2024-09-07 21:23:37,860]-INFO-84-Directory Created at artifacts/prepare_base_model
58889256/58889256 [==============================] - 8s 0us/step
[2024-09-07 21:23:49,902]-WARNING-359-Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.


d:\DL MODEL\ImageClassificationModel\cnnmodel\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     